In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import re

In [2]:
df = pd.read_csv('test.csv')
dft = pd.read_csv('todasCenario1.csv')

In [3]:
numero_linhas = df.shape[0]

print(f"O número de linhas no DataFrame é: {numero_linhas}")

O número de linhas no DataFrame é: 3490


In [4]:
import pandas as pd
import re

# Carregue o DataFrame 'df'

# SEATS __________________________________________________
# Check for model cars with missing values in the 'seats' column
distinct_full_names_with_blank_seats = df[df['seats'].isnull()]['full_name'].unique()
print('Distinct full names with blank seats:')
for full_name in distinct_full_names_with_blank_seats:
    print(full_name)

# Update number of seats based on conditions
df.loc[df['full_name'].isin(['2009 Maruti Estilo LXI', '2019 Nissan Kicks Diesel', '2010 Maruti Estilo LXI']), 'seats'] = 5
df.loc[df['full_name'] == '2008 Ford Endeavour 3.0 4x4 Thunder Plus', 'seats'] = 7

# Fill NaN values in 'seats' with 0 instead of dropping rows
df['seats'].fillna(0, inplace=True)

# ENGINE __________________________________________________
# Remove cc from engine_capacity
df['engine_capacity'] = df['engine_capacity'].str.replace('cc', '')

# FUEL TYPE __________________________________________________
# unique 'fuel_type'
valores_unicos_fuel_type = df['fuel_type'].unique()
print()
print(valores_unicos_fuel_type)

# KMS __________________________________________________
# remove kms from kms_driven
df['kms_driven'] = df['kms_driven'].str.replace('Kms', '')
# Remove , and blanks from 'kms_driven'
df['kms_driven'] = df['kms_driven'].str.replace(',', '').str.strip()

# MAX POWER __________________________________________________
# Convert to bhp - code for correcting max_power
#convert to bhp
Correct_Max_Power = []
for power in df['max_power']:
    if pd.notna(power):
        if isinstance(power, str):
            if 'bhp' in power:
                num_bhp = float(re.search(r'\d+.?\d', power).group())
                Correct_Max_Power.append(num_bhp)
            elif 'PS' in power:
                num_ps = float(re.search(r'\d+.?\d', power).group())
                num_bhp = round(num_ps * 0.98632, 2)
                Correct_Max_Power.append(num_bhp)
            else:
                num = re.search(r'\d+\.?\d*', power)
                Correct_Max_Power.append(float(num.group()) if num else None)
        else:
            Correct_Max_Power.append(power)
    else:
        Correct_Max_Power.append(None)


# Convert not NaN to float
Correct_Max_Power = [float(value) if value is not None else value for value in Correct_Max_Power]

df['max_power'] = Correct_Max_Power
# OWNER TYPE __________________________________________________
# Replace NaN values in 'owner_type' with 0 instead of dropping rows
df['owner_type'].fillna(0, inplace=True)

# Check
unique_values = df['owner_type'].unique()
print()
print(unique_values)

# INSURANCE __________________________________________________
# Delete "insurance"
df['insurance'] = df['insurance'].str.replace('insurance', '')

# Assign value to blank lines
df['insurance'].fillna('No Insurance', inplace=True)

# Blank space on "Third Party" lines
df['insurance'] = df['insurance'].str.replace('Third Party ', 'Third Party')

# Remove integer values
df = df[~df['insurance'].astype(str).str.isnumeric()]

# MILEAGE __________________________________________________
# Assign value to blank lines
df['mileage'] = df['mileage'].fillna(0)

# Remove and convert 'kmpl' and 'km/kg' from mileage
df = df[df['mileage'].notna()]
consumo_correto = []

for i in df['mileage']:
    match = re.search(r'(\d+\.\d+|\d+)', str(i))  # Encontrar números (com ou sem ponto decimal)
    
    if match:
        valor = float(match.group())
        
        if 'km/kg' in str(i):
            valor *= 1.40  # Converter de km/kg para kmpl
        
        consumo_correto.append(valor)

df['mileage'] = consumo_correto

# Restante do código...



Distinct full names with blank seats:
2005 Honda City 1.5 EXI
2009 Maruti Estilo LXI
2012 Honda Jazz Select Edition

['Petrol' 'Diesel' 'CNG' 'Electric' 'LPG']

['First Owner' 'Second Owner' 'Third Owner' 0 'Fourth Owner' 'Fifth Owner']


In [5]:
numero_linhas = df.shape[0]

print(f"O número de linhas no DataFrame é: {numero_linhas}")

O número de linhas no DataFrame é: 3490


In [6]:
#FullName ______________________________________

#ANO
# Extract the year and create a new column
df['ano'] = df['full_name'].str.extract(r'(\d{4})')

# Display the resulting DataFrame
print(df['ano'])

#MARCA
# Extract the first word after the year and create a new column
df['marca'] = df['full_name'].str.extract(r'\d{4}\s(\w+)')

# Print the resulting DataFrame
print(df['marca'])

#MODELO
# Get everything that comes after "marca"
df['modelo'] = df['full_name'].str.extract(r'\d{4}\s\w+\s(.+)')

# Print the resulting DataFrame
print(df['modelo'])

0       2017
1       2020
2       2010
3       2010
4       2018
        ... 
3485    2022
3486    2017
3487    2017
3488    2016
3489    2017
Name: ano, Length: 3490, dtype: object
0       Hyundai
1        Maruti
2        Maruti
3        Maruti
4       Hyundai
         ...   
3485       Tata
3486    Renault
3487      Honda
3488      Honda
3489     Maruti
Name: marca, Length: 3490, dtype: object
0       Grand i10 1.2 Kappa Sportz Option
1                           Swift AMT VXI
2                     Zen Estilo LX BSIII
3                                Alto LXi
4                     i20 Active S Petrol
                      ...                
3485                     Tigor EV XZ Plus
3486                         KWID 1.0 RXT
3487                       City i VTEC VX
3488                    Jazz 1.2 V i VTEC
3489                            Swift VXI
Name: modelo, Length: 3490, dtype: object


In [7]:
#BODY TYPE ----------------------
#Replace the missing types and the ones that were wrong
df.loc[df['full_name'].isin(['2009 Maruti Estilo LXI', '2019 Nissan Kicks Diesel', '2010 Maruti Estilo LXI', '2019 Maruti Wagon R ZXI AMT 1.2BSIV',
                             '2016 Mercedes-Benz B Class B180 Sport', '2016 Mercedes-Benz A Class A200 D Sport Edition',
                             '2004 Tata Indigo Marina GLX', '2017 Volvo V40 D3 R Design']), 'body_type'] = 'Hatchback'
df.loc[df['full_name'].isin(['2005 Tata Sumo Victa EX 10/7 Str BSII', '2008 Tata Sumo CX', '2010 Tata New Safari DICOR 2.2 EX 4x2', 
                             '2017 Mahindra KUV 100 mFALCON G80 K8', '2019 Mahindra Bolero Camper GOLD ZX 2WD']), 'body_type'] = 'SUV'
df.loc[df['full_name'].isin(['2015 Honda Mobilio E i-DTEC', '2018 Maruti Eeco 5 Seater AC BSIV', '2016 Datsun GO Plus A']), 'body_type'] = 'MUV'
df.loc[df['full_name'].isin(['2021 BMW 7 Series 730Ld DPE Signature','2006 Hyundai Accent CRDi', '2007 Maruti SX4 Vxi BSIII',
                             '2020 Mercedes-Benz S-Class Maybach S560', '2004 Skoda Octavia 1.8 Turbo Petrol RS MT']), 'body_type'] = 'Sedan'
df.loc[df['full_name'].isin(['2012 Mercedes-Benz SLK-Class SLK 350', '2013 Mercedes-Benz E-Class E350 Cabriolet', 
                             '2009 Mercedes-Benz SLK-Class 55 AMG']), 'body_type'] = 'Convertibles'
df.loc[df['full_name'] == '2011 Jaguar XK R V8 Coupe', 'body_type'] = 'Coupe'
df.loc[df['full_name'].isin(['2015 Chevrolet Enjoy 1.4 LS 7', '2004 Maruti Versa DX2 8-SEATER BSIII TWIN A/C', '2021 Maruti Eeco 5 STR With AC Plus HTR CNG',
                             '2022 Maruti Eeco 5 Seater AC 2020-2022', '2022 Maruti Eeco 5 Seater AC BSVI', '2017 Maruti Eeco Flexi Green', 
                             '2013 Maruti Omni Limited Edition', '2021 Maruti Eeco CNG 5 Seater AC', '2019 Maruti Eeco CNG 5 Seater AC BSIV', 
                             '2019 Maruti Eeco 5 Seater AC', '2013 Maruti Eeco 5 Seater Standard BSIV', '2015 Maruti Omni CNG', 
                             '2015 Maruti Omni MPI CARGO BSIV', '2016 Maruti Eeco CNG 5 Seater AC BSIV', '2013 Toyota Innova 2.5 E Diesel MS 7-seater',
                             '2015 Chevrolet Enjoy 1.4 LS 7']), 'body_type'] = 'Minivans'
df.loc[df['full_name'].isin(['2014 Tata Xenon XT EX 4X2', '2019 Isuzu D-Max High']), 'body_type'] = 'Pickup'

# Check
distinct_full_names_with_blank_body_type = df[df['body_type'].isnull()]['full_name'].unique()
print('Distinct full names with blank body_type:')
for full_name in distinct_full_names_with_blank_body_type:
    print(full_name)

Distinct full names with blank body_type:


In [8]:
# Mudar formatos

df = df.astype({
    'seats': float,
    'engine_capacity': float,
    'kms_driven': float,
    'max_power': float,
    'ano': float,
    'marca': 'string',
    'modelo': 'string',
    'body_type': 'string',
    'mileage': float,
})

In [9]:
# Importe a biblioteca pandas, se ainda não estiver importada
import pandas as pd

# Substitua os valores NaN por 'Indefinido' nas colunas específicas, exceto 'engine_capacity'
columns_to_replace = ['kms_driven', 'marca', 'modelo']
df[columns_to_replace] = df[columns_to_replace].fillna('Indefinido')

# Substitua os valores NaN na coluna 'engine_capacity' por 0
df['engine_capacity'] = df['engine_capacity'].fillna(0)
# Substitua os valores NaN na coluna 'max_power' por 0
df['max_power'] = df['max_power'].fillna(0)

In [10]:
# Change formats

df = df.astype({
    'seats': float,
    'engine_capacity': float,
    'kms_driven': float,
    'max_power': float,
    'ano': float,
    'marca': 'string',
    'modelo': 'string',
   
})

In [11]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['transmission_type'], dft['id_transmissao']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_transmissao'] = df['transmission_type'].map(mapeamento_max_power_group)

In [12]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['marca'], dft['id_marcas']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_marcas'] = df['marca'].map(mapeamento_max_power_group)

In [13]:
import pandas as pd

def adicionar_0_1_maior_valor(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_categoria_marca_dft = dft['id_marcas'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_categoria_marca_dft + 0.1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_marcas'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valor(df, dft)

# Visualizar o DataFrame 'df' atualizado

In [14]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['body_type'], dft['id_body_type']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_body_type'] = df['body_type'].map(mapeamento_max_power_group)

In [15]:
import pandas as pd

def adicionar_0_1_maior_valor(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_categoria_marca_dft = dft['id_body_type'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_categoria_marca_dft + 0.1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_body_type'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valor(df, dft)

# Visualizar o DataFrame 'df' atualizado

In [16]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['city'], dft['id_city']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_city'] = df['city'].map(mapeamento_max_power_group)

In [17]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['insurance'], dft['id_insurance']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_insurance'] = df['insurance'].map(mapeamento_max_power_group)

In [18]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['fuel_type'], dft['id_fuel_type']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_fuel_type'] = df['fuel_type'].map(mapeamento_max_power_group)

In [19]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['owner_type'], dft['id_owner_type']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_owner_type'] = df['owner_type'].map(mapeamento_max_power_group)

In [20]:
df = df.astype({
    
    'ano': float,
  
    
  
})

In [21]:
contagem_nulos = df.isnull().sum()

print(f"O número de valores nulos na coluna é: {contagem_nulos}")

O número de valores nulos na coluna é: full_name             0
registered_year      15
engine_capacity       0
insurance             0
transmission_type     0
kms_driven            0
owner_type            0
fuel_type             0
max_power             0
seats                 0
mileage               0
body_type             0
city                  0
ID                    0
ano                   0
marca                 0
modelo                0
id_transmissao        0
id_marcas             0
id_body_type          0
id_city               0
id_insurance          0
id_fuel_type          0
id_owner_type         9
dtype: int64


In [22]:
numero_linhas = df.shape[0]

print(f"O número de linhas no DataFrame é: {numero_linhas}")

O número de linhas no DataFrame é: 3490


In [23]:
df

,full_name,registered_year,engine_capacity,insurance,transmission_type,kms_driven,owner_type,fuel_type,max_power,seats,...,ano,marca,modelo,id_transmissao,id_marcas,id_body_type,id_city,id_insurance,id_fuel_type,id_owner_type
0,2017 Hyundai Grand i10 1.2 Kappa Sportz Option,Jun 2017,1197.0,Comprehensive,Manual,53658.0,First Owner,Petrol,81.8,5.0,...,2017.0,Hyundai,Grand i10 1.2 Kappa Sportz Option,0.131359,0.677665,0.364112,0.922710,0.445408,0.180334,0.150465
1,2020 Maruti Swift AMT VXI,Feb 2021,1197.0,Not Available,Automatic,33055.0,Second Owner,Petrol,81.8,5.0,...,2020.0,Maruti,Swift AMT VXI,0.583102,0.541440,0.364112,1.281577,1.418644,0.180334,0.620328
2,2010 Maruti Zen Estilo LX BSIII,Jul 2010,998.0,Third Party,Manual,90385.0,Third Owner,Petrol,67.1,5.0,...,2010.0,Maruti,Zen Estilo LX BSIII,0.131359,0.541440,0.364112,1.593888,0.256633,0.180334,1.365236
3,2010 Maruti Alto LXi,2010,796.0,Third Party,Manual,110000.0,First Owner,Petrol,46.3,5.0,...,2010.0,Maruti,Alto LXi,0.131359,0.541440,0.364112,0.767700,0.256633,0.180334,0.150465
4,2018 Hyundai i20 Active S Petrol,Oct 2018,1197.0,Comprehensive,Manual,58000.0,Second Owner,Petrol,81.8,5.0,...,2018.0,Hyundai,i20 Active S Petrol,0.131359,0.677665,0.364112,0.767700,0.445408,0.180334,0.620328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485,2022 Tata Tigor EV XZ Plus,2022,0.0,Third Party,Automatic,30000.0,First Owner,Electric,73.7,5.0,...,2022.0,Tata,Tigor EV XZ Plus,0.583102,1.230837,0.578053,1.096102,0.256633,2.455816,0.150465
3486,2017 Renault KWID 1.0 RXT,Apr 2017,999.0,Third Party,Manual,64024.0,First Owner,Petrol,67.0,5.0,...,2017.0,Renault,KWID 1.0 RXT,0.131359,1.419493,0.364112,0.868320,0.256633,0.180334,0.150465
3487,2017 Honda City i VTEC VX,Dec 2017,1497.0,Comprehensive,Manual,39000.0,First Owner,Petrol,117.3,5.0,...,2017.0,Honda,City i VTEC VX,0.131359,0.976238,0.578053,0.767700,0.445408,0.180334,0.150465
3488,2016 Honda Jazz 1.2 V i VTEC,Jun 2016,1199.0,Comprehensive,Manual,33000.0,First Owner,Petrol,88.7,5.0,...,2016.0,Honda,Jazz 1.2 V i VTEC,0.131359,0.976238,0.364112,0.922710,0.445408,0.180334,0.150465


In [24]:
df.columns

Index(['full_name', 'registered_year', 'engine_capacity', 'insurance',
       'transmission_type', 'kms_driven', 'owner_type', 'fuel_type',
       'max_power', 'seats', 'mileage', 'body_type', 'city', 'ID', 'ano',
       'marca', 'modelo', 'id_transmissao', 'id_marcas', 'id_body_type',
       'id_city', 'id_insurance', 'id_fuel_type', 'id_owner_type'],
      dtype='object')

In [25]:
#Export
#Not the target
colunas_selecionadas = ['id_marcas', 'ano', 'engine_capacity', 'id_insurance', 'id_transmissao', 'kms_driven', 'id_owner_type', 'id_fuel_type', 'max_power', 'seats', 'mileage', 'id_body_type', 'id_city', 'ID']


if set(colunas_selecionadas).issubset(df.columns):
    df_selecionado = df[colunas_selecionadas]
    df_selecionado.to_csv('TestCenario1.csv', index=False)